# Measuring the performance of Value_at_timestamp

In [21]:
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 

tqdm.pandas()

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

In [22]:
cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

In [23]:
mmsi_list

[(5322,),
 (2190045,),
 (2444044,),
 (9112856,),
 (9134186,),
 (84877508,),
 (97000071,),
 (109080281,),
 (111219502,),
 (111219512,),
 (111219516,),
 (111219520,),
 (111232512,),
 (111232523,),
 (219001275,),
 (111257004,),
 (111257010,),
 (111261504,),
 (111264243,),
 (111265132,),
 (111265558,),
 (111265586,),
 (111265587,),
 (111503001,),
 (203123400,),
 (203245382,),
 (203274200,),
 (203504300,),
 (205032000,),
 (205144000,),
 (205146000,),
 (210512000,),
 (205240000,),
 (205250270,),
 (205256900,),
 (205258754,),
 (205262810,),
 (205283000,),
 (205340000,),
 (205353330,),
 (205435790,),
 (205465000,),
 (205475000,),
 (205534790,),
 (205904810,),
 (205555000,),
 (205594000,),
 (205642000,),
 (205665130,),
 (205770000,),
 (205910210,),
 (209072000,),
 (209078000,),
 (209114000,),
 (210520000,),
 (210523000,),
 (211448760,),
 (209144000,),
 (209151000,),
 (209183000,),
 (209207000,),
 (209216000,),
 (209217000,),
 (209221000,),
 (209239000,),
 (209247000,),
 (209296000,),
 (20931300

In [24]:
cursor.execute("SELECT * FROM public.PyMEOS_demo WHERE MMSI = 308654000 ;")
mmsi, trajectory, sog = cursor.fetchone()
trajectory

TGeogPointSeq([POINT(10.4322 57.8445)@2023-06-01 00:00:00+02, POINT(10.4312 57.8445)@2023-06-01 00:00:06+02, POINT(10.4305 57.8445)@2023-06-01 00:00:12+02, POINT(10.4287 57.8445)@2023-06-01 00:00:24+02, POINT(10.4271 57.8445)@2023-06-01 00:00:36+02, POINT(10.4264 57.8445)@2023-06-01 00:00:42+02, POINT(10.4264 57.8445)@2023-06-01 00:00:44+02, POINT(10.4247 57.8445)@2023-06-01 00:00:53+02, POINT(10.4239 57.8444)@2023-06-01 00:01:00+02, POINT(10.4229 57.8444)@2023-06-01 00:01:06+02, POINT(10.4222 57.8444)@2023-06-01 00:01:12+02, POINT(10.4212 57.8444)@2023-06-01 00:01:18+02, POINT(10.4204 57.8444)@2023-06-01 00:01:24+02, POINT(10.4197 57.8444)@2023-06-01 00:01:30+02, POINT(10.4187 57.8444)@2023-06-01 00:01:36+02, POINT(10.418 57.8444)@2023-06-01 00:01:42+02, POINT(10.4162 57.8443)@2023-06-01 00:01:54+02, POINT(10.4155 57.8443)@2023-06-01 00:02:00+02, POINT(10.4145 57.8443)@2023-06-01 00:02:06+02, POINT(10.4138 57.8443)@2023-06-01 00:02:11+02, POINT(10.4129 57.8443)@2023-06-01 00:02:18+02,

In [25]:
timestamps = trajectory.timestamps()
timestamps

[datetime.datetime(2023, 6, 1, 0, 0, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 6, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 12, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 24, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 36, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 44, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 0, 53, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 6, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 12, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 18, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 24, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 30, tzinfo=tzoffset(None, 7200)),
 datetime.dateti

# Evaluating performance of Value_at_timestamp

In [26]:
import time

times = []
for dt in timestamps:
    now = time.time()
    trajectory.value_at_timestamp(dt)
    times.append(time.time() - now)
print(f"Average time: {sum(times) / len(times)}")

Average time: 3.314498651920503e-05


In [27]:
import time

times = []

values = trajectory.values()
for i in range(len(timestamps)):
    now = time.time()
    val = values[i]
    times.append(time.time() - now)
print(f"Average time: {sum(times) / len(times)}")

Average time: 1.323070533374404e-07


In [32]:
import inspect

print(inspect.signature(trajectory.temporal_sample))

(duration: 'Union[str, timedelta]', start: 'Optional[Union[str, datetime]]' = None) -> 'TG'


In [34]:
a = trajectory.temporal_sample(start=timestamps[0],duration= timedelta(minutes=1))
a

TGeogPointSeq({POINT(10.4322 57.8445)@2023-06-01 00:00:00+02, POINT(10.4239 57.8444)@2023-06-01 00:01:00+02, POINT(10.4155 57.8443)@2023-06-01 00:02:00+02, POINT(10.407114285714284 57.84420000030816)@2023-06-01 00:03:00+02, POINT(10.398742857142855 57.844100000481475)@2023-06-01 00:04:00+02, POINT(10.390414285714284 57.84400000030815)@2023-06-01 00:05:00+02, POINT(10.382042857142853 57.84390000048149)@2023-06-01 00:06:00+02, POINT(10.373642857142853 57.8438000004815)@2023-06-01 00:07:00+02, POINT(10.365242857142853 57.84370000048146)@2023-06-01 00:08:00+02, POINT(10.356842857142853 57.8436000004815)@2023-06-01 00:09:00+02, POINT(10.3484 57.8435)@2023-06-01 00:10:00+02, POINT(10.34 57.8434)@2023-06-01 00:11:00+02, POINT(10.3316 57.8433)@2023-06-01 00:12:00+02, POINT(10.3232 57.8432)@2023-06-01 00:13:00+02, POINT(10.3147 57.8431)@2023-06-01 00:14:00+02, POINT(10.3063 57.843)@2023-06-01 00:15:00+02, POINT(10.2978 57.8429)@2023-06-01 00:16:00+02, POINT(10.289424999696367 57.84221250043007)

In [33]:
print(inspect.getsource(trajectory.temporal_sample))

    def temporal_sample(
        self,
        duration: Union[str, timedelta],
        start: Optional[Union[str, datetime]] = None,
    ) -> TG:
        """
        Returns a new :class:`Temporal` downsampled with respect to ``duration``.

        Args:
            duration: A :class:`str` or :class:`timedelta` with the duration of
                the temporal tiles.
            start: A :class:`str` or :class:`datetime` with the start time of
                the temporal tiles. If None, the start time used by default is
                Monday, January 3, 2000.
        MEOS Functions:
            temporal_tsample
        """
        if start is None:
            st = pg_timestamptz_in("2000-01-03", -1)
        elif isinstance(start, datetime):
            st = datetime_to_timestamptz(start)
        else:
            st = pg_timestamptz_in(start, -1)
        if isinstance(duration, timedelta):
            dt = timedelta_to_interval(duration)
        else:
            dt = pg_interval_

In [14]:
dir(trajectory)

['BaseClass',
 'ComponentClass',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_factory',
 '_inner',
 '_is_protocol',
 '_mobilitydb_name',
 '_parse_function',
 '_projection_cache',
 'always_equal',
 'always_not_equal',
 'angular_difference',
 'append_instant',
 'append_sequence',
 'as_ewkt',
 'as_geojson',
 'as_hexwkb',
 'as_mfjson',
 'as_wkb',
 'as_wkt',
 'at',
 'at_max',
 'at_min',
 'azimuth',
 'bearing',
 'bounding_box',
 'contains',
 'cumulative_length',
 'delete',
 'direction',
 'disjo

In [20]:
trajectory.stboxes()

[STBox(SRID=4326;GEODSTBOX XT(((10.4312,57.8445),(10.4322,57.8445)),[2023-06-01 00:00:00+02, 2023-06-01 00:00:06+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4305,57.8445),(10.4312,57.8445)),[2023-06-01 00:00:06+02, 2023-06-01 00:00:12+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4287,57.8445),(10.4305,57.8445)),[2023-06-01 00:00:12+02, 2023-06-01 00:00:24+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4271,57.8445),(10.4287,57.8445)),[2023-06-01 00:00:24+02, 2023-06-01 00:00:36+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4264,57.8445),(10.4271,57.8445)),[2023-06-01 00:00:36+02, 2023-06-01 00:00:42+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4264,57.8445),(10.4264,57.8445)),[2023-06-01 00:00:42+02, 2023-06-01 00:00:44+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4247,57.8445),(10.4264,57.8445)),[2023-06-01 00:00:44+02, 2023-06-01 00:00:53+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4239,57.8444),(10.4247,57.8445)),[2023-06-01 00:00:53+02, 2023-06-01 00:01:00+02])),
 STBox(SRID=4326;GEODSTBOX XT(((10.4229,

In [16]:
trajectory.values()

[<POINT (10.432 57.844)>,
 <POINT (10.431 57.844)>,
 <POINT (10.43 57.844)>,
 <POINT (10.429 57.844)>,
 <POINT (10.427 57.844)>,
 <POINT (10.426 57.844)>,
 <POINT (10.426 57.844)>,
 <POINT (10.425 57.844)>,
 <POINT (10.424 57.844)>,
 <POINT (10.423 57.844)>,
 <POINT (10.422 57.844)>,
 <POINT (10.421 57.844)>,
 <POINT (10.42 57.844)>,
 <POINT (10.42 57.844)>,
 <POINT (10.419 57.844)>,
 <POINT (10.418 57.844)>,
 <POINT (10.416 57.844)>,
 <POINT (10.416 57.844)>,
 <POINT (10.414 57.844)>,
 <POINT (10.414 57.844)>,
 <POINT (10.413 57.844)>,
 <POINT (10.412 57.844)>,
 <POINT (10.411 57.844)>,
 <POINT (10.41 57.844)>,
 <POINT (10.41 57.844)>,
 <POINT (10.408 57.844)>,
 <POINT (10.407 57.844)>,
 <POINT (10.406 57.844)>,
 <POINT (10.406 57.844)>,
 <POINT (10.404 57.844)>,
 <POINT (10.404 57.844)>,
 <POINT (10.403 57.844)>,
 <POINT (10.402 57.844)>,
 <POINT (10.401 57.844)>,
 <POINT (10.4 57.844)>,
 <POINT (10.399 57.844)>,
 <POINT (10.398 57.844)>,
 <POINT (10.397 57.844)>,
 <POINT (10.396 57.

In [17]:
trajectory

TGeogPointSeq([POINT(10.4322 57.8445)@2023-06-01 00:00:00+02, POINT(10.4312 57.8445)@2023-06-01 00:00:06+02, POINT(10.4305 57.8445)@2023-06-01 00:00:12+02, POINT(10.4287 57.8445)@2023-06-01 00:00:24+02, POINT(10.4271 57.8445)@2023-06-01 00:00:36+02, POINT(10.4264 57.8445)@2023-06-01 00:00:42+02, POINT(10.4264 57.8445)@2023-06-01 00:00:44+02, POINT(10.4247 57.8445)@2023-06-01 00:00:53+02, POINT(10.4239 57.8444)@2023-06-01 00:01:00+02, POINT(10.4229 57.8444)@2023-06-01 00:01:06+02, POINT(10.4222 57.8444)@2023-06-01 00:01:12+02, POINT(10.4212 57.8444)@2023-06-01 00:01:18+02, POINT(10.4204 57.8444)@2023-06-01 00:01:24+02, POINT(10.4197 57.8444)@2023-06-01 00:01:30+02, POINT(10.4187 57.8444)@2023-06-01 00:01:36+02, POINT(10.418 57.8444)@2023-06-01 00:01:42+02, POINT(10.4162 57.8443)@2023-06-01 00:01:54+02, POINT(10.4155 57.8443)@2023-06-01 00:02:00+02, POINT(10.4145 57.8443)@2023-06-01 00:02:06+02, POINT(10.4138 57.8443)@2023-06-01 00:02:11+02, POINT(10.4129 57.8443)@2023-06-01 00:02:18+02,

In [7]:
a = trajectory.instants()
a

[TGeogPointInst(POINT(10.4322 57.8445)@2023-06-01 00:00:00+02),
 TGeogPointInst(POINT(10.4312 57.8445)@2023-06-01 00:00:06+02),
 TGeogPointInst(POINT(10.4305 57.8445)@2023-06-01 00:00:12+02),
 TGeogPointInst(POINT(10.4287 57.8445)@2023-06-01 00:00:24+02),
 TGeogPointInst(POINT(10.4271 57.8445)@2023-06-01 00:00:36+02),
 TGeogPointInst(POINT(10.4264 57.8445)@2023-06-01 00:00:42+02),
 TGeogPointInst(POINT(10.4264 57.8445)@2023-06-01 00:00:44+02),
 TGeogPointInst(POINT(10.4247 57.8445)@2023-06-01 00:00:53+02),
 TGeogPointInst(POINT(10.4239 57.8444)@2023-06-01 00:01:00+02),
 TGeogPointInst(POINT(10.4229 57.8444)@2023-06-01 00:01:06+02),
 TGeogPointInst(POINT(10.4222 57.8444)@2023-06-01 00:01:12+02),
 TGeogPointInst(POINT(10.4212 57.8444)@2023-06-01 00:01:18+02),
 TGeogPointInst(POINT(10.4204 57.8444)@2023-06-01 00:01:24+02),
 TGeogPointInst(POINT(10.4197 57.8444)@2023-06-01 00:01:30+02),
 TGeogPointInst(POINT(10.4187 57.8444)@2023-06-01 00:01:36+02),
 TGeogPointInst(POINT(10.418 57.8444)@20

In [5]:
from datetime import datetime, timedelta 
# keys = number of instants, values = time it took to perform value at timestamp for the given timestamp

timestamp = datetime(2023, 6 , 1, 0, 0, 0)

times = {}

for mmsi in mmsi_list:
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {mmsi[0]} ;")
    mmsi, trajectory, sog = cursor.fetchone()
    now = datetime.now()
    try:
        val = trajectory.value_at_timestamp(timestamp)
        times[trajectory.num_instants()] = datetime.now() - now
    except:
        continue

times

{334: datetime.timedelta(microseconds=1219),
 8671: datetime.timedelta(microseconds=297),
 4728: datetime.timedelta(microseconds=154),
 8751: datetime.timedelta(microseconds=215),
 954: datetime.timedelta(microseconds=169),
 8019: datetime.timedelta(microseconds=198),
 5685: datetime.timedelta(microseconds=226),
 228: datetime.timedelta(microseconds=117),
 1137: datetime.timedelta(microseconds=165),
 2901: datetime.timedelta(microseconds=163),
 5357: datetime.timedelta(microseconds=194),
 1970: datetime.timedelta(microseconds=179),
 59: datetime.timedelta(microseconds=169),
 1934: datetime.timedelta(microseconds=161),
 6260: datetime.timedelta(microseconds=202),
 2322: datetime.timedelta(microseconds=133),
 2868: datetime.timedelta(microseconds=295),
 6178: datetime.timedelta(microseconds=244),
 2062: datetime.timedelta(microseconds=236),
 3584: datetime.timedelta(microseconds=212),
 384: datetime.timedelta(microseconds=190),
 4078: datetime.timedelta(microseconds=238),
 1452: datetime

In [7]:
# show values of keys and values of times ordered by values
sorted(times.items(), key=lambda x: x[1], reverse=True)

[(334, datetime.timedelta(microseconds=1219)),
 (8382, datetime.timedelta(microseconds=463)),
 (7891, datetime.timedelta(microseconds=405)),
 (7465, datetime.timedelta(microseconds=405)),
 (1888, datetime.timedelta(microseconds=400)),
 (10249, datetime.timedelta(microseconds=396)),
 (2433, datetime.timedelta(microseconds=385)),
 (8100, datetime.timedelta(microseconds=384)),
 (10666, datetime.timedelta(microseconds=353)),
 (15282, datetime.timedelta(microseconds=347)),
 (7155, datetime.timedelta(microseconds=345)),
 (9947, datetime.timedelta(microseconds=342)),
 (4945, datetime.timedelta(microseconds=341)),
 (12568, datetime.timedelta(microseconds=335)),
 (12758, datetime.timedelta(microseconds=328)),
 (8022, datetime.timedelta(microseconds=322)),
 (2849, datetime.timedelta(microseconds=304)),
 (4221, datetime.timedelta(microseconds=303)),
 (10055, datetime.timedelta(microseconds=301)),
 (8671, datetime.timedelta(microseconds=297)),
 (14541, datetime.timedelta(microseconds=296)),
 (3202

In [17]:

connection.commit()
cursor.close()
pymeos_finalize()

# Implementing double queue for FPS

In [8]:
from collections import deque

# Create a deque with a maximum length of 10
deque_size = 10
dq = deque(maxlen=deque_size)

# Function to add an element to the deque
def add_element(dq, element):
    dq.append(element)
    print(f"Added {element}: {list(dq)}")

# Sample usage
for i in range(15):
    add_element(dq, i)


Added 0: [0]
Added 1: [0, 1]
Added 2: [0, 1, 2]
Added 3: [0, 1, 2, 3]
Added 4: [0, 1, 2, 3, 4]
Added 5: [0, 1, 2, 3, 4, 5]
Added 6: [0, 1, 2, 3, 4, 5, 6]
Added 7: [0, 1, 2, 3, 4, 5, 6, 7]
Added 8: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Added 9: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Added 10: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Added 11: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Added 12: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Added 13: [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Added 14: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [11]:
sum(dq)/len(dq)

9.5